# Ouverture des données générées

In [4]:
# Import du fichier d'exemples

def get_raw_data_go():
    ''' Returns the set of samples from the local file or download it if it does not exists'''
    import gzip, os.path
    import json

    raw_samples_file = "samples-9x9.json.gz"

    if not os.path.isfile(raw_samples_file):
        print("File", raw_samples_file, "not found, I am downloading it...", end="")
        import urllib.request 
        urllib.request.urlretrieve ("https://www.labri.fr/perso/lsimon/static/inge2-ia/samples-9x9.json.gz", "samples-9x9.json.gz")
        print(" Done")

    with gzip.open("samples-9x9.json.gz") as fz:
        data = json.loads(fz.read().decode("utf-8"))
    return data

data = get_raw_data_go()
print("We have", len(data),"examples")

We have 41563 examples


In [5]:
import copy
import numpy as np
import random
import time
letters_to_index = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'H':7, 'J':8}

index_to_letters = {0:'A', 1:'B', 2:'C', 3:'D', 4:'E', 5:'F', 6:'G', 7:'H', 8:'J'}

def integer_to_letter(i):
    return index_to_letters[i]

def name_to_coord(s):
    assert s != "PASS"

    col = letters_to_index[s[0]]
    lin = int(s[1:]) - 1
    return col, lin

def coord_to_name(s):
    col = integer_to_letter(s[0])
    return col + str(s[1]+1)

def rotate_coords_90(coords):
    rotated_coords = []
    for row in range(9):
        for col in range(9):
            # Get the coordinates for the current position
            coord = (row, col)
            # Compute the rotated coordinates
            rotated_coord = (8-col, row)
            # If the original coordinates are in the input list, add the rotated coordinates
            if coord in coords:
                rotated_coords.append(rotated_coord)
    if 'PASS' in coords:
        rotated_coords.append('PASS')
    return rotated_coords

def name_list_to_coord(mov):
    coords = []
    for m in mov:
        if (m != 'PASS'):
            coords.append(name_to_coord(m))
        else:
            coords.append(m)
    return coords

def coord_list_to_name(coord):
    name = []
    for c in coord:
        if (c != 'PASS'):
            name.append(coord_to_name(c))
        else:
            name.append('PASS')
    return name

def rotate_list(name_list):
    coords = name_list_to_coord(name_list)
    rotated_coords = rotate_coords_90(coords)
    return coord_list_to_name(rotated_coords)

def rotate_sample(sample):
    # Rotate the positions of the stones and the moves
    sample['list_of_moves']=rotate_list(sample['list_of_moves'])
    sample['black_stones'] = rotate_list(sample['black_stones'])
    sample['white_stones'] = rotate_list(sample['white_stones'])
    return sample

# we copy the original data given earlier in another array
datacopy = copy.deepcopy(data)
# to have all 4 rotations
for i in range (3):
    print("Rotation numéro " + str(i+1) + "/3")
    for j in range (len(datacopy)):
        sample = datacopy[j]
        sample = rotate_sample(sample)
        datacopy[j] = sample
    data = np.append(data,datacopy)
# we now add vertical rotations
# we had theses:
# 12  31  43  24
# 34  42  21  13
# we will have theses in addition:
# 21  13  34  42
# 43  24  12  31
datacopy = copy.deepcopy(data)
#current_time = int(time.time())
#np.random.seed(current_time)
#random.shuffle(data)

np.random.seed(42) #this is not a joke: i found out this is a good seed.
random.shuffle(data) 
#Warning : list of moves will not be right ordered. Doesn't really matter for me since i don't use it but here we are...
print("We now have", len(data),"examples")

Rotation numéro 1/3
Rotation numéro 2/3
Rotation numéro 3/3
We now have 166252 examples


In [7]:
import numpy as np
import time

X = []
y = []

for sample in data:
    input_data = np.zeros((11, 11, 2))
    for i in range(9):
        for j in range(9):
            pos = coord_to_name((i,j))
            if pos in sample['black_stones']:
                input_data[i+1][j][0] = 1
            elif pos in sample['white_stones']:
                input_data[i+1][j][1] = 1

    # add an extra binary feature to indicate which player is next in the game
    #if sample['player'] == 'black':
    #    player_turn = 1
    #else:
    #    player_turn = 0
    player_turn = 1
    X.append(np.append(input_data, player_turn))
    y.append([sample['black_wins'] / sample['rollouts'],sample['white_wins']/sample['rollouts']])
    
X = np.array(X)
y = np.array(y)

# split into data set and validation data set
split_idx = len(X) // 2
X_train, y_train = X[:split_idx], y[:split_idx]
X_val, y_val = X[split_idx:], y[split_idx:]
print("Data sorted done.")

Data sorted done.


In [14]:
import numpy as np
import time

X = []
y = []

for sample in data:
    input_data = np.zeros((3, 11, 11))  # change the order of dimensions
    for i in range(9):
        for j in range(9):
            pos = coord_to_name((i,j))
            if pos in sample['black_stones']:
                input_data[0][i+1][j] = 1  # assign the first channel
            elif pos in sample['white_stones']:
                input_data[1][i+1][j] = 1  # assign the second channel

    # set the player turn channel
    input_data[2] = 0  # assign the third channel with 0

    X.append(input_data)
    y.append([sample['black_wins'] / sample['rollouts'], sample['white_wins'] / sample['rollouts']])    
    
X = np.array(X)
y = np.array(y)

# split into data set and validation data set
split_idx = len(X) // 2
X_train, y_train = X[:split_idx], y[:split_idx]
X_val, y_val = X[split_idx:], y[split_idx:]

from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, Dropout, BatchNormalization, Activation, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), input_shape=(3, 11, 11), activation='relu'))  # change the input shape
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

learning_rate_default = 0.001
# this will slow down the learning rate
def scheduler(epoch):
    if epoch < 1:
        return learning_rate_default
    else:
        return learning_rate_default * np.exp(0.1 * (1 - epoch))

model.compile(optimizer=Adam(lr=learning_rate_default), loss='mse', metrics=['mae','mse'])
lr_scheduler = LearningRateScheduler(scheduler)

model.summary()

history = model.fit(X_train, y_train, epochs=12, batch_size=64, validation_data=(X_val, y_val), callbacks=[lr_scheduler])


ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv2d_9. Consider increasing the input size. Received input shape [None, 1, 9, 64] which would produce output shape with a zero or negative value in a dimension.

# Last step

Prepare your model to predict the set of new data to predict, you will have only 6 hours to push your predictions.

(may be you would like to express, when guessing the percentage of wins for blacks, that it should reflect the fact that this score should be the same for all the symmetries you considered)...

In [7]:
def position_predict(black_stones, white_stones):
    input_data = np.zeros((11, 11, 2))
    for i in range(9):
        for j in range(9):
            pos = coord_to_name((i,j))
            if pos in black_stones:
                input_data[i+1][j][0] = 1
            elif pos in white_stones:
                input_data[i+1][j][1] = 1
    input_data = np.expand_dims(input_data, axis=0)
    prediction = model.predict(input_data)[0] 
    
    return prediction

import random
import time

#seed = int(round(time.time() * 1000))
#random.seed(seed)
#nb = 1000
#avg = 0
## try out for nb values if my average meet what i need
#for i in range(nb):
#    j = random_int = random.randint(0, len(X)-1)
#    prediction = position_predict(data[j]["black_stones"], data[j]["white_stones"])
#    avg= avg + abs(prediction - data[j]["black_wins"]/data[j]["rollouts"])
#print(avg/nb)

# Ainsi, pour le rendu, en admettant que newdata soit le fichier json contenant les nouvelles données que 
# l'on vous donnera 24h avant la fin, vous pourrez construire le fichier resultat ainsi

def create_result_file(newdata):
    ''' Exemple de méthode permettant de générer le fichier de resultats demandés. '''
    resultat  = [position_predict(d["black_stones"], d["white_stones"]) for d in newdata]
    with open("my_predictions.txt", "w") as f:
         for p in resultat:
            f.write(str(p)+"\n")



1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 7ms/step


1/1 [==============================] - 0s 7ms/step
[0.05755739]
